In [1]:
from pathlib import Path
from wsireg.wsireg import WsiReg2D

## wsireg for multi-modal image registration of whole slide images

First, we will initialize the graph model and add modalities. Each modality is loaded with 3
pieces of information:
* the file path to image file to be loaded
* the image spatial resolution,
* the preprocessing parameters.

Images that are to be registered must be a single plane and
the preprocessing parameters take care of this.

Two default processes are available:
* _FL_ : take multi-channel image and converts to single channel through maximum intensity projection
* _BF_ : assumed to be 24-bit RGB images, these are converted to a single channel by taking RGB to greyscale
then intensity is inverted so background is black and foreground is white.


In [2]:
# we define a project name, output directory and whether to cache
# preprocessed images when making a registration graph instance
# project name = Sample107
# output directory is same as the data
# cache_images is True, this saves time during registration and avoids unnecessary io
reg_graph = WsiReg2D("ExampleReg","./", cache_images=True)

We will add multiple image modalities (7 in total) and 6 of these will be transformed to a single target but will take different paths.
The target modality is a post-imaging mass spectrometry(IMS) autofluorescence (AF) image to which IMS data has been previously registered.
Other modalities are AF of this section before IMS, AF on a serial section prior to immunofluorescence (IF). Finally, 3 cycles
of IF on a single section.

In [3]:
# global target modality
# preprocessing
# ch_indicices selects certain channels
# as_uint8 bytescales data to unsigned 8 bit for memory saving
# contrast_enhance makes low contrast fluorescence images brighter for registration
# if a value in the processing dict is set to None it is skipped. These are the defaults

reg_graph.add_modality(
    "postAF_IMS",
    "/data/SharedData/biomic/registration/S107_postAF_IMS.tif",
    image_res=0.5,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": None,
        "as_uint8": True,
        "contrast_enhance": None,
    },
    channel_names=["eGFP - Post-IMS Autofluorescence"],
    channel_colors=["white"],
)

In [4]:
# this is an AF modality
# we select the first channel for registration
# scale intensity to uint8
# and rotate the image 180 degrees counter-clockwise using "rot_cc"
# very large rotations are hard to capture with intensity based registration
# therefore we use prior knowledge to add this information

reg_graph.add_modality(
    "preAF_IMS",
    "/data/SharedData/biomic/registration/S107_preAF_IMS.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 180,
    },
    channel_names=[
        "Autofluorescence - DAPI",
        "Autofluorescence - eGFP",
        "Autofluorescence - dsRed",
    ],
    channel_colors=["blue", "green", "red"],
)




In [5]:
# serial section AF

# in addition the previous modalities pre-processing
# this modality's section is "mirror" to the previous and thus needs
# a coordinate flip so we use "flip" preprocessing and flip horizontally the image

reg_graph.add_modality(
    "preAF_MxIF",
    "/data/SharedData/biomic/registration/S107_preAF_MxIF.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
    channel_names=[
        "Autofluorescence - DAPI",
        "Autofluorescence - eGFP",
        "Autofluorescence - dsRed",
    ],
    channel_colors=["blue", "green", "red"],
)

### PAS registration
This modality contains a histological stain with RGB pixel data that was performed on the IMS section after IMS.

In [6]:
# PAS stain
# here we set image_type to "BF"
# this will invert image intensity values so background is 0 to better match
# the other photometric qualities of the other modalities
reg_graph.add_modality(
    "PAS_IMS",
    "/data/SharedData/biomic/registration/S107_PAS.scn",
    image_res=0.5,
    prepro_dict={
        "image_type": "BF",
        "ch_indices": None,
        "as_uint8": True,
        "rot_cc": 90,
        "contrast_enhance": None,
    },
    channel_names=["R", "G", "B",],
    channel_colors=["red", "green", "blue"],
)

### IF registration
The multiple cycles of IF are loaded as modalities selecting their DAPI channel to achieve cell nucleus-to-nucleus
registration. However, MxIF cycle 1 (modality : MxIF_cyc1) will be registered to this sections corresponding AF image.
Because of this, we will use special preprocessing for that registration described later.

In [7]:
# all MxIF cycles have similar spatial pre-processing

reg_graph.add_modality(
    "MxIF_cyc1",
    "/data/SharedData/biomic/registration/S107_MxIF_Cyc1.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
    channel_names=[
        "DAPI - Hoechst (Nuclei)",
        "FITC - Aquaporin 1, AQP1 (Proximal Tubules and desc. thin limb)",
        "Cy3 -  NaCl Co-transporter, NCC (Distal convoluted tubule)",
        "Cy5 - Synaptopodin (Glomerular epithelium)",
    ],
    channel_colors=["blue", "green", "red", "yellow"],
)

reg_graph.add_modality(
    "MxIF_cyc2",
    "/data/SharedData/biomic/registration/S107_MxIF_Cyc2.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
    channel_names=[
        "DAPI - Hoechst (Nuclei)",
        "FITC - Aquaporin 1, AQP1 (Proximal Tubules and desc. thin limb)",
        "Cy3 - Tamm Horsfall protein, THP (thick limb)",
        "Cy5 - Aquaporin 2 (collecting ducts)",
    ],
    channel_colors=["blue", "green", "red", "yellow"],
)

reg_graph.add_modality(
    "MxIF_cyc3",
    "/data/SharedData/biomic/registration/S107_MxIF_Cyc3.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
    channel_names=[
        "DAPI - Hoechst (Nuclei)",
        "FITC - Laminin, gamma-1 (Basement membrane)",
        "Cy3 - Tamm Horsfall protein, THP (thick limb)",
        "Cy5 - EpCAM (Distal tubule and thick limb)",
    ],
    channel_colors=["blue", "green", "red", "yellow"],
)


### defining the registrations (edges) of the graph

Now we will define how modalities move through the graph. For every modality to be registered we define
three settings:
* modality to be registered
* modality to which it will be registered
* through modalities, modalities which will form the path of alignment from one image to another
* registration parameters (elastix parameters)
* whether to use special preprocessing for this particular registration

For this case where there are serial sections, we want to define registration paths that use the best information
for alignment. For instance, we have AF images on the two serial sections so complex non-linear registration is best performed mono-modally.
The alternative would be to register the IF data directly to the AF image, where the specificty of IF would not help find commonalities between the modalities
on which to register.

In [8]:
# register preAF_IMS to postAF_IMS
reg_graph.add_reg_path(
    "preAF_IMS", "postAF_IMS", thru_modality=None, reg_params=["rigid"]
)


# register preAF_MxIF to postAF_IMS
# BUT go through preAF_IMS (same section)
# this will align preAF_MXiF to preAF_IMS then
# append the transformations from preAF_IMS to postAF_IMS
# in this cases we use both rigid and non-linear ("nl") elastix transformation models
# for default we have "rigid","affine" and "nl" these work in 90% of the cases
# alternatively you can add a file path string to elastix registration parameters saved in a text file
reg_graph.add_reg_path(
    "preAF_MxIF", "postAF_IMS", thru_modality="preAF_IMS", reg_params=["rigid", "nl"],
)

# register PAS_IMS to postAF_IMS through preAF_IMS
reg_graph.add_reg_path(
    "PAS_IMS", "postAF_IMS", thru_modality="preAF_IMS", reg_params=["rigid"],
)

In [9]:
# here we register MxIF_cyc1 to postAF_IMS through preAF_MxIF
# the graph will find the necessary edges to follow to get from MxIF_cyc1 to postAF_IMS
# but we use special preprocessing here since the set preprocessing for MxIF_cyc1 is
# to select the DAPI nuclei channel but here we will use the maximum intensity projection
# to process the 4 channels to 1 to align to AF
# this increases the amount of texture and spatial information for registration
# as DAPI is spatially sparse in signal, compared to all channels

reg_graph.add_reg_path(
    "MxIF_cyc1",
    "postAF_IMS",
    thru_modality="preAF_MxIF",
    reg_params=["rigid"],
    override_prepro={
        "source": {
            "image_type": "FL",
            "ch_indices": None,
            "as_uint8": True,
            "rot_cc": 90,
            "flip": "h",
        },
        "target": None,
    },
)



In [10]:
# now we align MxIF cycles 2 and 3 to postAF, this time going through MxIF Cyc1
# MxIF_cyc1 goes through preAF_MxIF to preAF_IMS to postAF_IMS
reg_graph.add_reg_path(
    "MxIF_cyc2", "postAF_IMS", thru_modality="MxIF_cyc1", reg_params=["rigid"],
)

reg_graph.add_reg_path(
    "MxIF_cyc3", "postAF_IMS", thru_modality="MxIF_cyc1", reg_params=["rigid"],
)

### graph information
We can use some utility functions to learn about the graph.

In [11]:
print("number of modalities : {}".format(reg_graph.n_modalities))
print("number of registrations : {}".format(reg_graph.n_registrations))

number of modalities : 7
number of registrations : 6


In [12]:
print(reg_graph.reg_paths)

{'preAF_IMS': ['postAF_IMS'], 'preAF_MxIF': ['preAF_IMS', 'postAF_IMS'], 'PAS_IMS': ['preAF_IMS', 'postAF_IMS'], 'MxIF_cyc1': ['preAF_MxIF', 'postAF_IMS'], 'MxIF_cyc2': ['MxIF_cyc1', 'postAF_IMS'], 'MxIF_cyc3': ['MxIF_cyc1', 'postAF_IMS']}


In [13]:
# show all transformation paths
print(reg_graph.transform_paths)


{'preAF_IMS': [{'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'preAF_MxIF': [{'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'PAS_IMS': [{'source': 'PAS_IMS', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'MxIF_cyc1': [{'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}, {'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'MxIF_cyc2': [{'source': 'MxIF_cyc2', 'target': 'MxIF_cyc1'}, {'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}, {'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'MxIF_cyc3': [{'source': 'MxIF_cyc3', 'target': 'MxIF_cyc1'}, {'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}, {'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}]}


In [14]:
# look at how MxIF_cyc3 will be transformed specifically
print(*reg_graph.transform_paths["MxIF_cyc1"], sep="\n")



{'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}
{'source': 'preAF_MxIF', 'target': 'preAF_IMS'}
{'source': 'preAF_IMS', 'target': 'postAF_IMS'}


From the above we can see a list of transformations that will be applied to transform MxIF_cyc1 to
postAF_IMS.

### executing the graph
Now we have loaded and set all registration modalities, we can begin the alignment and transformation process.

In [15]:
# align images, this can take some time depending on computer performance
# generally 3-10 minutes per registration when using the default models
reg_graph.register_images()


performing preprocessing:  max_int_proj
cannot perform maximum intensity project on single channel image
performing preprocessing:  contrast_enhance
performing preprocessing:  max_int_proj
cannot perform maximum intensity project on single channel image
no inversions to compute
performing preprocessing:  max_int_proj
cannot perform maximum intensity project on single channel image
performing preprocessing:  contrast_enhance
no inversions to compute
performing preprocessing:  inv_int
no inversions to compute
performing preprocessing:  max_int_proj
performing preprocessing:  contrast_enhance
no inversions to compute
performing preprocessing:  max_int_proj
cannot perform maximum intensity project on single channel image
performing preprocessing:  contrast_enhance
performing preprocessing:  max_int_proj
cannot perform maximum intensity project on single channel image
performing preprocessing:  contrast_enhance
no inversions to compute
performing preprocessing:  max_int_proj
cannot perform 

In [16]:
# save the transformations as a json on disk (can be reloaded and used again)
reg_graph.save_transformations()

In [17]:
# transform and save images to disk as zarr directory stores for remote viewing
# this function returns a list of file paths to the zarr stores created during transformation
# this list can then be passed into vizarr
zarr_fps = reg_graph.transform_images(file_writer="zarr")

transforming preAF_IMS to postAF_IMS
transforming preAF_MxIF to postAF_IMS
transforming PAS_IMS to postAF_IMS
transforming MxIF_cyc1 to postAF_IMS
transforming MxIF_cyc2 to postAF_IMS
transforming MxIF_cyc3 to postAF_IMS
transforming non-registered modality : postAF_IMS 


## Vizarr Imjoy Plugin setup

For more details, please see: https://github.com/hms-dbmi/vizarr

In [41]:
from imjoy import api
import zarr

def encode_zarr_store(zobj):
    def getItem(key):
        return zobj.store[key]

    def setItem(key, value):
        zobj.store[key] = value

    def containsItem(key):
        if key in zobj.store:
            return True

    return {
        "_rintf": True,
        "_rtype": 'zarr-array' if isinstance(zobj, zarr.Array) else 'zarr-group',
        "getItem": getItem,
        "setItem": setItem,
        "containsItem": containsItem,
    }

api.registerCodec({'name': 'zarr-array', 'type': zarr.Array, "encoder": encode_zarr_store})
api.registerCodec({'name': 'zarr-group', 'type': zarr.Group, "encoder": encode_zarr_store})

class VivPlugin:
    def __init__(self, images):
        if not isinstance(images, list):
            images = [images]
        self.images = images

    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(
            type="vizarr",
            src="https://hms-dbmi.github.io/vizarr/"
        )
        for img in self.images:
            await viewer.add_image(img)


In [ ]:
def run_vizarr(image):
    api.export(VivPlugin(image))

## gather the data for vizarr

In [38]:
registered_images = [
    {
        "source": zarr.open(zfp),
        "name": Path(zfp).stem.replace("ExampleReg-", "").split("_to")[0],
    }
    for zfp in zarr_fps
]


In [51]:
run_vizarr(registered_images)

<IPython.core.display.Javascript object>

## check how nuclei overlay accross MxIF cycles
Examine only the DAPI stains of the MxIF images to see nuclear overlap.

In [49]:
import json
from zarr.util import json_dumps

class StackedOMEStore:
    def __init__(self, z_grps, axis=0):
        self.z_grps = z_grps
        self._stack_axis = axis
        self._proxy_meta = self._create_proxy_meta()

    def __getitem__(self, key):
        if key.endswith(".zarray"):
            return self._proxy_meta[key]

        if key.endswith(".zgroup") or key.endswith(".zattrs"):
            # let first store handle all other requests for metadata
            return self.z_grps[0].store[key]

        # Should just be multiscale pattern now
        # Probably a better way to do this...
        path = Path(key)
        ckeys = path.name.split(".")
        z_grp_key = int(ckeys[self._stack_axis])
        zgrp = self.z_grps[z_grp_key]

        ckeys[self._stack_axis] = "0" # change key to zero in underling store
        return zgrp.store[str(path.parent / ".".join(ckeys))]

    def __setitem__(self, key, value):
        raise NotImplementedError()

    def _create_proxy_meta(self):
        proxy = dict()
        grp = self.z_grps[0]
        for d in grp.attrs["multiscales"][0]["datasets"]:
            path = f"{d['path']}/.zarray"
            b = grp.store[path]
            arr_meta = json.loads(b)
            arr_meta["shape"][self._stack_axis] = len(self.z_grps)
            proxy[path] = json_dumps(arr_meta)
        return proxy

    def __iter__(self):
        for key in self.z_grps[0].store:
            yield key

# get MxIF data
dapi_grps = [zarr.open(zfp)
    for zfp in zarr_fps if "MxIF_cyc" in Path(zfp).stem
]

# stack MxIF images along first axis
dapi_stack = zarr.open(StackedOMEStore(dapi_grps, axis=0))

# prepare for vizarr with specific channel names
dapi_registered = { "source": dapi_stack, 
                   "channel_axis": 0,
                   "name":"DAPI merged",
                   "names": ["DAPI Cycle 1", "DAPI Cycle 2", "DAPI Cycle 3"],  
                   
                  }

In [50]:
run_vizarr(dapi_registered)

<IPython.core.display.Javascript object>